In [47]:
import openai, json
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

messages = []

In [48]:
def get_weather(city):
    return '30도.'

def get_people(city):
    return '5천만명.'

FUNCTION_MAP = {
    'get_weather': get_weather,
    'get_people': get_people
}

In [49]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",
                    },
                },
                "required": ["city"],
            },
        },
	},
    {
        "type": "function",
        "function": {
            "name": "get_people",
            "description": "A function to get the people of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the people of.",
                    },
                },
                "required": ["city"],
            },
        },
	},
]

In [50]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):

    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args {arguments} for a result of {result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [51]:
while True:
    message = input("Send a message to the LLM...")
    
    if message == "quit" or message == "q":
        break
    
    messages.append({
        "role": "user",
        "content": message
    })
    print(f"User: {message}")
    
    call_ai()

User: My name is Jeongmok
AI: Nice to meet you, Jeongmok! How can I assist you today?
User: What is my name
AI: Your name is Jeongmok.
User: What is the weather in Korea
Calling function: get_weather with {"city":"Seoul"}
Ran get_weather with args {'city': 'Seoul'} for a result of 30도.
AI: The weather in Seoul, Korea is 30 degrees Celsius. Would you like to know more about the weather or anything else?
User: How many people in Korea
Calling function: get_people with {"city":"Seoul"}
Ran get_people with args {'city': 'Seoul'} for a result of 5천만명.
AI: The population of South Korea is approximately 50 million people. If you need more information, feel free to ask!
